# Project 1

We need to design and implement a class that will be used to represent bank accounts

We want the fallowing functionalities and characteristics:
- accounts are auniquely identified by an account number (assume it will be passed in the initializer)
- account holders  have a first name and last name
- accounts have an associated preferred time zone offset(e.g. -7 for MST)
- balances need to be zero or higher, and should not be directly suitable
- deposits and withdrawals can me be made (given sufficient founds)
    - If a withdrawal is attempted would result in negative founds, the transaction should be declined
- a monthly interest rate exists and is applicable to all accounts uniformly. There should be a method that can be called to calculare the interest on the current balance using the current insterest rate, and add it to the balance

In [13]:
# Imports
from uuid import uuid4
from datetime import timedelta, datetime
import numbers
import pytz
import unittest
import itertools
from abc import ABC, abstractmethod

In [2]:
class TimeZone:

    def __init__(self, *, name, offset_hours, offset_minutes):
        # name should be non empty string
        if name is None or len(str(name).strip()) == 0:
            raise ValueError('Timezone name cannot be empty.')

        # offset_hours should be integer.
        if not isinstance(offset_hours, numbers.Integral):
            raise ValueError('Hour offset must be an integer.')
        
        # offset_minutes should be integer.
        if not isinstance(offset_minutes, numbers.Integral):
            raise ValueError('Minute offset must be an integer.')
        
        # offset_minutes should be between -59 and 59.
        if offset_minutes > 59 or offset_minutes < -59:
            raise ValueError('Minutes offset must be between -59 and 59 (inclusive).')
        
        # offset_hours should be between -12 and 14.
        offset = timedelta(hours=offset_hours, minutes=offset_minutes)
        if offset < timedelta(hours=-12, minutes=0) or offset > timedelta(hours=14, minutes=0):
            raise ValueError('Offset must be between -12:00 and +14:00.')
        
        self._name = str(name).strip()
        self._offset_hours = offset_hours
        self._offset_minutes = offset_minutes
        self._offset = offset

    @property
    def offset(self):
        return self._offset
    
    @property
    def name(self):
        return self._name
    
    def __repr__(self):
        return f'<TimeZone(name={self.name}, offset_hours={self._offset_hours}, offset_minutes={self._offset_minutes})>'
    
    def __str__(self):
        return self.name
    
    def __eq__(self, other):
        return (isinstance(other, TimeZone) and
                self.name == other.name and
                self._offset_hours == other._offset_hours and
                self._offset_minutes == other._offset_minutes)
    
    def __hash__(self):
        return hash((self.name, self._offset_hours, self._offset_minutes))

In [20]:
class TestTimezone(unittest.TestCase):

    def test_timezone_valid_input(self):
        tz = TimeZone(name='ABC', offset_hours=3, offset_minutes=30)
        self.assertEqual(tz.name, 'ABC')
        self.assertEqual(tz.offset, timedelta(hours=3, minutes=30))
    
    def test_timezone_invalid_input(self):
        with self.assertRaises(ValueError):
            tz = TimeZone(name=None, offset_hours=3, offset_minutes=30)

        with self.assertRaises(ValueError):
            tz = TimeZone(name=" ", offset_hours=3, offset_minutes=30)
        
        with self.assertRaises(ValueError):
            tz = TimeZone(name='ABC', offset_hours=3.5, offset_minutes=30)
        
        with self.assertRaises(ValueError):
            tz = TimeZone(name='ABC', offset_hours=3, offset_minutes=60)
        
        with self.assertRaises(ValueError):
            tz = TimeZone(name='ABC', offset_hours=15, offset_minutes=30)
        
        with self.assertRaises(ValueError):
            tz = TimeZone(name='ABC', offset_hours=-13, offset_minutes=30)
        
        with self.assertRaises(ValueError):
            tz = TimeZone(name='ABC', offset_hours=3, offset_minutes=-60)
    
    def test_timezone_repr(self):
        tz = TimeZone(name='ABC', offset_hours=3, offset_minutes=30)
        self.assertEqual(repr(tz), '<TimeZone(name=ABC, offset_hours=3, offset_minutes=30)>')
    
    def test_timezone_str(self):
        tz = TimeZone(name='ABC', offset_hours=3, offset_minutes=30)
        self.assertEqual(str(tz), 'ABC')
    
    def test_timezone_eq(self):
        tz1 = TimeZone(name='ABC', offset_hours=3, offset_minutes=30)
        tz2 = TimeZone(name='ABC', offset_hours=3, offset_minutes=30)
        self.assertEqual(tz1, tz2)
    
    def test_timezone_hash(self):
        tz1 = TimeZone(name='ABC', offset_hours=3, offset_minutes=30)
        tz2 = TimeZone(name='ABC', offset_hours=3, offset_minutes=30)
        self.assertEqual(hash(tz1), hash(tz2))


In [7]:
def transaction_id_generator():
    while True:
        yield f"trans_{uuid4()}_{datetime.now().year}"

In [18]:
class TestTransactionIdGenerator(unittest.TestCase):

    def test_transaction_id_generator(self):
        gen = transaction_id_generator()
        id1 = next(gen)
        id2 = next(gen)
        self.assertNotEqual(id1, id2)
        self.assertTrue(id1.startswith('trans_'))
        self.assertTrue(id2.startswith('trans_'))
        self.assertTrue(id1.endswith(f'_{datetime.now().year}'))
        self.assertTrue(id2.endswith(f'_{datetime.now().year}'))


In [ ]:
class Holder:

    def __init__(self, *, first_name: str, last_name: str, dob: str):
        """ Holder class to store holder information. 
        
        Args:
            first_name (str): First name of the holder.
            last_name (str): Last name of the holder.
            dob (str): Date of birth of the holder in YYYY-MM-DD format.

        Raises:
            ValueError: If first_name or last_name is empty.
            ValueError: If dob is not in YYYY-MM-DD format.
        """
        self._holder_id = str(uuid4())
        self.first_name = first_name
        self.last_name = last_name
        self.dob = dob

    @property
    def holder_id(self):
        return self._holder_id
    
    @property
    def first_name(self):
        return self._first_name
    
    @property
    def last_name(self):
        return self._last_name
    
    @property
    def dob(self):
        return self._dob
    
    @first_name.setter
    def first_name(self, value):
        self._first_name = Holder.validate_name(value, 'First name')

    @last_name.setter
    def last_name(self, value):
        self._last_name = Holder.validate_name(value, 'Last name')

    @dob.setter
    def dob(self, value):
        self._dob = Holder.validate_dob(value)
    
    @staticmethod
    def validate_name(value: str, label: str) -> str:
        if value is None or len(str(value).strip()) == 0:
            raise ValueError(f'{label} cannot be empty.')
        return str(value).strip()
    
    @staticmethod
    def validate_dob(value: str) -> str:
        try:
            datetime.strptime(value, '%Y-%m-%d')
        except ValueError:
            raise ValueError('DOB must be in YYYY-MM-DD format.')
        return value
    
    def __repr__(self) -> str:
        return f"<Holder(first_name={self._first_name}, last_name={self._last_name}, dob={self._dob})>"
    
    def __str__(self) -> str:
        return f"{self._first_name} {self._last_name}"

In [17]:
class TestHolder(unittest.TestCase):

    def test_holder_valid_input(self):
        holder = Holder(first_name='John', last_name='Doe', dob='1990-01-01')
        self.assertEqual(holder.first_name, 'John')
        self.assertEqual(holder.last_name, 'Doe')
        self.assertEqual(holder.dob, '1990-01-01')
    
    def test_holder_invalid_input(self):
        with self.assertRaises(ValueError):
            holder = Holder(first_name=None, last_name='Doe', dob='1990-01-01')
        
        with self.assertRaises(ValueError):
            holder = Holder(first_name=' ', last_name='Doe', dob='1990-01-01')
        
        with self.assertRaises(ValueError):
            holder = Holder(first_name='John', last_name=None, dob='1990-01-01')
        
        with self.assertRaises(ValueError):
            holder = Holder(first_name='John', last_name=' ', dob='1990-01-01')

        with self.assertRaises(ValueError):
            holder = Holder(first_name='John', last_name='Doe', dob='1990-01-32')

        with self.assertRaises(ValueError):
            holder = Holder(first_name='John', last_name='Doe', dob='1990-13-01')

        with self.assertRaises(ValueError):
            holder = Holder(first_name='John', last_name='Doe', dob='1990-01-01 00:00:00')
    
    def test_holder_str(self):
        holder = Holder(first_name='John', last_name='Doe', dob='1990-01-01')
        self.assertEqual(str(holder), 'John Doe')

    def test_holder_repr(self):
        holder = Holder(first_name='John', last_name='Doe', dob='1990-01-01')
        self.assertEqual(repr(holder), '<Holder(first_name=John, last_name=Doe, dob=1990-01-01)>')
        

In [14]:
class Account(ABC):

    transaction_counter = itertools.count(100)

    def __init__(
            self, 
            *, 
            account_id: str, 
            holder: Holder, 
            currency: str = 'USD',  
            initial_balance: numbers.Number = 0,
            tz: TimeZone = None
        ):
        
        if not isinstance(holder, Holder):
            raise ValueError('Holder must be a valid Holder object')
        
        self._account_id = account_id
        self._holder = holder
        self._balance = float(initial_balance)
        self._currency = currency
        
        if tz is None:
            self.timezone = TimeZone(name='UTC', offset_hours=0, offset_minutes=0)
        else:
            self.timezone = tz

    @property
    def account_id(self):
        return self._account_id

    @property
    def balance(self):
        return self._balance
    
    @property
    def holder_first_name(self):
        return self._holder.first_name
    
    @property
    def holder_last_name(self):
        return self._holder.last_name
    
    @property
    def timezone(self):
        return self._tz
    
    @timezone.setter
    def timezone(self, value):
        if not isinstance(value, TimeZone):
            raise ValueError('Timezone must be a valid TimeZone object')
        self._tz = value

    def _set_balance(self, value: numbers.Number):
        if not isinstance(value, numbers.Number):
            raise NotImplemented
        total = self._balance + value
        if total < 0:
            raise ValueError("Insufficient founds")
        self._balance = total

    def __repr__(self) -> str:
        return f"{self._holder.first_name} {self._holder.last_name} - {self._account_id} - {self._currency} {self._balance}"
    
    def withdraw(self, amount: numbers.Number):
        if not isinstance(amount, numbers.Number) or amount < 0:
            raise NotImplemented
        self._set_balance(-amount)
        return amount

    def deposit(self, amount: numbers.Number):
        if not isinstance(amount, numbers.Number) or amount < 0:
            raise NotImplemented
        self._set_balance(amount)
        return self.balance
    
    @abstractmethod
    def calculate_interest(self):
        return NotImplemented

        
class BasicAccount(Account):
    _interest_rate = 0.2

    @classmethod
    def get_interest_rate(cls):
        return cls._interest_rate
    
    @classmethod
    def set_interest_rate(cls, value):
        if not isinstance(value, numbers.Real) or value < 0:
            raise ValueError('Interest rate must be a positive number')
        cls._interest_rate = value

    def calculate_interest(self):
        pass

In [16]:
class TestInterestRateInBasicAccount(unittest.TestCase):

    def test_interest_rate_in_basic_account(self):
        self.assertEqual(BasicAccount.get_interest_rate(), 0.2)
        BasicAccount.set_interest_rate(0.3)
        self.assertEqual(BasicAccount.get_interest_rate(), 0.3)

    def tearDown(self) -> None:
        BasicAccount.set_interest_rate(0.2)
        return super().tearDown()
    

In [21]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_holder_invalid_input (__main__.TestHolder.test_holder_invalid_input) ... ok
test_holder_repr (__main__.TestHolder.test_holder_repr) ... ok
test_holder_str (__main__.TestHolder.test_holder_str) ... ok
test_holder_valid_input (__main__.TestHolder.test_holder_valid_input) ... ok
test_interest_rate_in_basic_account (__main__.TestInterestRateInBasicAccount.test_interest_rate_in_basic_account) ... ok
test_timezone_eq (__main__.TestTimezone.test_timezone_eq) ... ok
test_timezone_hash (__main__.TestTimezone.test_timezone_hash) ... ok
test_timezone_invalid_input (__main__.TestTimezone.test_timezone_invalid_input) ... ok
test_timezone_repr (__main__.TestTimezone.test_timezone_repr) ... ok
test_timezone_str (__main__.TestTimezone.test_timezone_str) ... ok
test_timezone_valid_input (__main__.TestTimezone.test_timezone_valid_input) ... ok
test_transaction_id_generator (__main__.TestTransactionIdGenerator.test_transaction_id_generator) ... ok

---------------------------------------------------